## Initial setup

In [2]:
import importlib # Used to add reload functionality, for when we update our own custom libraries
import re
from typing import List
import ollama
import pandas as pd
import json
import os

# Custom libraries
import brikasutils as bu
importlib.reload(bu)
import shared_utils as utils
from shared_utils import systemMsg, userMsg, assistantMsg, find_most_similar
importlib.reload(utils)

<module 'shared_utils' from '/Users/twenythree/Home/Prot/Thesis/shared_utils.py'>

## Persona

In [12]:
import re

# Custom libraries
import standard_msg_reader as fb
importlib.reload(fb)
import persona
importlib.reload(persona)


<module 'persona' from '/Users/twenythree/Home/Prot/Thesis/persona.py'>

In [13]:
texts_with_elias = [
    "selected-data/elias/message_1.json",
]
texts_with_petyo = [
    "selected-data/petyo/message_1.json",
    "selected-data/petyo/message_2.json",
    "selected-data/petyo/message_3.json",
    "selected-data/petyo/message_4.json",
    "selected-data/petyo/message_5.json",
]
texts_with_others_dict = {
    "anna": ["selected-data/others/anna.json"],
    "patryk": ["selected-data/others/patryk.json"],
    "andreas": ["selected-data/others/andreas.json"],
    "victoria": ["selected-data/others/victoria.json", "selected-data/others/victoria2.json"],
    "joanna": ["selected-data/others/joanna.json"],
    "antoni": ["selected-data/others/antoni.json"],
    "arijan": ["selected-data/others/arijan.json"],
    "denis": ["selected-data/others/denis.json"],
    "alexandra": ["selected-data/others/alexandra.json"],
    "FED": ["selected-data/others/FED.json"],
    "filip": ["selected-data/others/filip.json"],
    "kuba": ["selected-data/others/kuba.json"],
    "laura": ["selected-data/others/laura.json"],
    "liisa": ["selected-data/others/liisa.json"],
    "luiza": ["selected-data/others/luiza.json"],
    "marcus": ["selected-data/others/marcus.json"],
}


ab = persona.PersonaEncoder()
ab.parse_fb_messages(texts_with_elias, "elias")
ab.parse_fb_messages(texts_with_petyo, "petyo")



for name, texts in texts_with_others_dict.items():
    ab.parse_fb_messages(texts, name)

ab.filter_chats_empty()
ab.filter_chats_regex(utils.BLACKLIST_CHAT_REGEX_FILTERS)

# Compress names
for nameid, chat in ab.chats.items():
    for msg in chat:  
        msg.sender = "Persona" if msg.sender == "Airidas Brikas" else "Friend"

# Start all chats from 2/3rds
for nameid, chat in ab.chats.items():
    ab.chats[nameid] = chat[int(len(chat)/3 * 2):]

# ab.select_chat_limited_by_tokens("elias", 6000)
# ab.select_chat_limited_by_tokens("petyo", 6000)
ab.select_chat_full("elias")
ab.select_chat_full("petyo")

for name in texts_with_others_dict.keys():
    ab.select_chat_full(name)

token_counts = ab.count_all_selected_chat_tokens() # token_counts used later for statistics
print(f"Combined tokens: {sum(token_counts.values())}")
persona_text = ab.output()
bu.quickTXT(persona_text, filename=f"ignorefolder/prompts")

Read 1946 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-09-13 to 2024-03-06
Messages saved to self.chats['elias']
Read 40036 messages from 5 files. Failed to read 0 messages.
Messages ranged from 2020-08-17 to 2024-03-04
Messages saved to self.chats['petyo']
Read 7953 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2023-05-12 to 2024-03-04
Messages saved to self.chats['anna']
Read 5734 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2020-09-18 to 2024-03-02
Messages saved to self.chats['patryk']
Read 372 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2022-03-19 to 2024-02-24
Messages saved to self.chats['andreas']
Read 3399 messages from 2 files. Failed to read 0 messages.
Messages ranged from 2021-08-23 to 2024-03-02
Messages saved to self.chats['victoria']
Read 2951 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2022-11-13 to 2024-02-19
Messages saved to self.ch

### Debug

In [ ]:
ab.chats["elias"][:60]

## Survey

In [14]:
import survey
importlib.reload(survey)
surv = survey.PersonalitySurvey()

Using default Personality Survey CSV file: surveys/survey_personality-test.csv


## Embedding

### Debug

Stored 'texts_with_elias' (list)


### Chunking

In [8]:
# PARAMETERS
EMBED_MODEL = "nomic-embed-text"        # nomic-embed-text = long ctx / mxbai-embed-large = big
CHUNK_SIZE = 30                         # Number of messages per chunk
OVERLAP_SIZE = 10                       # Number of overlapping messages between consecutive chunks

In [10]:
import ollama
import numpy as np
from numpy.linalg import norm

# different chunk size
chunks = []
stat_total_msgs_in_chunks = 0 # for statistics

# Iterate over chats and messages to create chunks
for chat in ab.selectedChats.values():
    messages = list(chat)  # Convert chat iterator to list for easier slicing
    num_messages = len(messages)

    # Create overlapping chunks of messages
    for i in range(0, num_messages - CHUNK_SIZE + 1, CHUNK_SIZE - OVERLAP_SIZE):
        chunk = messages[i:i + CHUNK_SIZE]  # Extract chunk of messages
        chunk_text = "\n".join(str(msg) for msg in chunk)  # Concatenate messages into a single string
        chunks.append(chunk_text)  # Append chunk to list of chunks

        stat_total_msgs_in_chunks += len(chunk) # For statistics


##### Display Info
total_messages = sum(len(chat) for chat in ab.selectedChats.values())
chunks_count = len(chunks)
avg_chunk_char_len = np.mean([len(chunk) for chunk in chunks])

print(
    f"Chunk count: {chunks_count}",
    f"Average chunk character length: {round( avg_chunk_char_len)}",
    f"Rough estimate of tokens per chunk: {round(avg_chunk_char_len / 4)} (4 characters per token)",
    f"Messagees in input count: {total_messages}",
    f"Messages in chunks count: {stat_total_msgs_in_chunks}",
    f"Chunk \ Input ratio: {round(stat_total_msgs_in_chunks / total_messages,2)} (OVERLAP_SIZE={OVERLAP_SIZE})",
    f"Chunk Python type: {type(chunks[0])}",
    sep="\n"
) 

Chunk count: 1112
Average chunk character length: 1575
Rough estimate of tokens per chunk: 394 (4 characters per token)
Messagees in input count: 22561
Messages in chunks count: 33360
Chunk \ Input ratio: 1.48 (OVERLAP_SIZE=10)
Chunk Python type: <class 'str'>


### Generate Embeddings 

In [11]:
########### Serialization ########
EMBEDDING_NAMEID = "airidas_reversed"
AUTO_INFO = {
    "model": EMBED_MODEL,
    "CHUNK_SIZE": CHUNK_SIZE,
    "OVERLAP_SIZE": OVERLAP_SIZE,
    "chunks_count": chunks_count,
    "total_messages": total_messages,
    "stat_total_msgs_in_chunks": stat_total_msgs_in_chunks,
    "modules_chat": token_counts,
}
##################################

# Generate embeddings for each chunk
embeddings = []

progress, chunks_len = 0, len(chunks) # for progress bar
for chunk_text in chunks:
    progress += 1
    print(f"Chunk {progress}/{chunks_len}")

    embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
    embeddings.append(embedding)


# Display and save results (if needed later)
print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

bu.if_dir_not_exist_make("embeddings")
bu.quickJSON(AUTO_INFO, f"embeddings/{EMBEDDING_NAMEID}_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"embeddings/{EMBEDDING_NAMEID}_embeddings.json")

Chunk 1/1112
Chunk 2/1112
Chunk 3/1112
Chunk 4/1112
Chunk 5/1112
Chunk 6/1112
Chunk 7/1112
Chunk 8/1112
Chunk 9/1112
Chunk 10/1112
Chunk 11/1112
Chunk 12/1112
Chunk 13/1112
Chunk 14/1112
Chunk 15/1112
Chunk 16/1112
Chunk 17/1112
Chunk 18/1112
Chunk 19/1112
Chunk 20/1112
Chunk 21/1112
Chunk 22/1112
Chunk 23/1112
Chunk 24/1112
Chunk 25/1112
Chunk 26/1112
Chunk 27/1112
Chunk 28/1112
Chunk 29/1112
Chunk 30/1112
Chunk 31/1112
Chunk 32/1112
Chunk 33/1112
Chunk 34/1112
Chunk 35/1112
Chunk 36/1112
Chunk 37/1112
Chunk 38/1112
Chunk 39/1112
Chunk 40/1112
Chunk 41/1112
Chunk 42/1112
Chunk 43/1112
Chunk 44/1112
Chunk 45/1112
Chunk 46/1112
Chunk 47/1112
Chunk 48/1112
Chunk 49/1112
Chunk 50/1112
Chunk 51/1112
Chunk 52/1112
Chunk 53/1112
Chunk 54/1112
Chunk 55/1112
Chunk 56/1112
Chunk 57/1112
Chunk 58/1112
Chunk 59/1112
Chunk 60/1112
Chunk 61/1112
Chunk 62/1112
Chunk 63/1112
Chunk 64/1112
Chunk 65/1112
Chunk 66/1112
Chunk 67/1112
Chunk 68/1112
Chunk 69/1112
Chunk 70/1112
Chunk 71/1112
Chunk 72/1112
C

### Load Embeddings From File (optional)

In [15]:
EMBEDDING_NAMEID = "airidas_reversed"

import json
with open(f"embeddings/{EMBEDDING_NAMEID}_embeddings.json", "r") as f:
    data = json.load(f)
    chunks = data["chunks"]
    embeddings = data["embeddings"]

try:
    with open(f"embeddings/{EMBEDDING_NAMEID}_info.json", "r") as f:
        AUTO_INFO = json.load(f)
        try:
            EMBED_MODEL = AUTO_INFO["model"]
        except KeyError:
            print("WARNING: Info text does not contain model information")
except:
    print("WARNING: No Info file found. Make sure embedding model is matching.")

print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

Chunks:1112, embeds:1112


#### debug

## Retrieval

### Static

In [39]:
import ollama
# RETRIEVAL_PROMPT = "video game features"
RETRIEVAL_PROMPT = "openess conciousness extrovert aggreableness neuroticism"
# RETRIEVAL_PROMPT = "personality"
CHUNKS_COUNT_IN_CTX = 12 # Number of nearby chunks to put in context window
# COMMENT 04-16, perhaps we could try 5x retrievals with isolated semantics

# Perform similarity search and print simulations
prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
chunks_most_similar_embeddings = find_most_similar(prompt_embedding, embeddings)[:CHUNKS_COUNT_IN_CTX]
chunks_most_similar = []
for embedding in chunks_most_similar_embeddings:
    chunks_most_similar.append(chunks[embedding[1]])

# token counts in all similar chunks
tokens_in_chunks = 0
for chunk in chunks_most_similar:
    tokens_in_chunks += utils.count_tokens(chunk)
print(f"Tokens in chunks: {tokens_in_chunks}")

bu.quickTXT("\n\n".join(chunks_most_similar), filename="ignorefolder/chunks.txt")



Tokens in chunks: 7702


### Dynamic

##### (dynamic, same as question)

In [16]:
dynamic_retrieval_prompts = list(surv.questions)

#### (dynamic, custom)

In [ ]:
# TODO

#### Run dynamic retrieval

In [17]:
CHUNKS_COUNT_IN_CTX = 12 # Number of nearby chunks to put in context window
dynamic_chunks_most_similar: List[List[str]] = [] 

progress = 0
lenn = len(dynamic_retrieval_prompts)
for prompt in dynamic_retrieval_prompts:
    progress += 1
    print(f"\rPrompt {progress}/{lenn}", end="")

    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]
    chunks_most_similar_embeddings = find_most_similar(prompt_embedding, embeddings)[:CHUNKS_COUNT_IN_CTX]
    chunks_most_similar = []
    for embedding in chunks_most_similar_embeddings:
        chunks_most_similar.append(chunks[embedding[1]])

    dynamic_chunks_most_similar.append(chunks_most_similar)
print(end="\n")
    
# VANITY PRINT
tokens_in_chunks = 0
for chunks_most_similar in dynamic_chunks_most_similar:
    for chunk in chunks_most_similar:
        tokens_in_chunks += utils.count_tokens(chunk)
    
print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
bu.quickJSON(dynamic_chunks_most_similar, filename=f"ignorefolder/dynamic-chunks.json")

Prompt 50/50
Tokens in average chunk group: 6885.88


#### Preview generated dynamic chunks (optional)

In [44]:
preview_text = ""
PREVIEW_LIMIT = 10

for i, chunks_most_similar in enumerate(dynamic_chunks_most_similar):
    preview_text += f"==============Prompt: {dynamic_retrieval_prompts[i]}==============\n"
    for j, chunk in enumerate(chunks_most_similar):
        if j >= PREVIEW_LIMIT:
            break
        preview_text += f"=======CHUNK {j}=======\n{chunk}\n\n"
    preview_text += "\n\n"
bu.quickTXT(preview_text, filename=f"ignorefolder/dynamic-chunks_preview.txt")

## Survey Simulation

### Prompt Builder - Persona - Dynamic

In [39]:
final_prompts = []

prompt_template = """
for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
    p = [
        systemMsg(
            "You are specialized in impersonating people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit tastes by shadowing chats between the subject and friends. You will be asked to answer questions from the point of view of the persona. Text below:",
            "Conversations between persona and friends",
            "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ),
        # Understanding affirmation
        assistantMsg('I will answer from the point of view of the persona, based on what I could the deduct from the text provided.'),
        # Survey question. With Simulation
        userMsg("\\n".join([
            f"Persona is surveyed about their personality. The persona must choose answer the question below with one of the given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option. ",
            question,
            "The persona chooses: "
        ])),
    ]
    final_prompts.append(p) 
"""

exec(prompt_template)

prompt_info = utils.describe_prompts_and_print(final_prompts)
bu.quickJSON(final_prompts, "ignorefolder/prompts.json")

Created 40 prompts.
Average prompt size: 7059 tokens.
Min prompt size: 4391, Max prompt size: 12647


In [40]:
prompt_template

'\nfor question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):\n    p = [\n        systemMsg(\n            "You are specialized in impersonating people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit tastes by shadowing chats between the subject and friends. You will be asked to answer questions from the point of view of the persona. Text below:",\n            "Conversations between persona and friends",\n            "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)\n        ),\n        # Understanding affirmation\n        assistantMsg(\'I will answer from the point of view of the persona, based on what I could the deduct from the text provided.\'),\n        # Survey question. With Simulation\n        userMsg("\\n".join([\n            f"Persona is surveyed about their personality. The persona must choose answer the question below with one of the given options: {\', \'.

### Prompt Builder - Persona

In [40]:
final_prompts = []

for question in surv.questions:
    p = [
        systemMsg(
            "You are specialized in impersonating people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit tastes by shadowing chats between the subject and friends. You will be asked to answer questions from the point of view of the persona. Text below:",
            "Conversations between persona and friends",
            "\nNEW CONVERSATION:\n".join(chunks_most_similar)
        ),
        # Understanding affirmation
        assistantMsg('I will answer from the point of view of the persona, based on what I could the deduct from the text provided.'),
        # Survey question. With Simulation
        userMsg("\n".join([
            f"Persona is surveyed about their personality. The persona must choose answer the question below with one of the given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option. ",
            question,
            "The persona chooses: "
        ])),
    ]
    final_prompts.append(p)

prompt_info = utils.describe_prompts_and_print(final_prompts) # Vanity print
bu.quickJSON(final_prompts, "ignorefolder/prompts.json")

Created 50 prompts.
Average prompt size: 7922 tokens.
Min prompt size: 7919, Max prompt size: 7927


### Prompt Builder - Base case

In [50]:
final_prompts = []

for question in surv.questions:
    p = [
        systemMsg(
            "You are participating in a survey. You will be presented with a series of questions about your personality.",
            f"You must choose answer to the question below with one of the five options: {', '.join(surv.POSSIBLE_ANSWERS)}. The answer must only contain the chosen option. "
        ),
        # Understanding affirmation
        assistantMsg('Understood. I will answer the question below with one of the given options.'),
        # Survey question. With Simulation
        userMsg(
            question,
            "Your choice: "
        ),
    ]
    final_prompts.append(p)

prompt_info = utils.describe_prompts_and_print(final_prompts) # Vanity print
bu.quickJSON(final_prompts, "ignorefolder/prompts.json")

Created 50 prompts.
Average prompt size: 94 tokens.
Min prompt size: 91, Max prompt size: 99


###### Unused / Debug

In [11]:
persona_text = "Favorite video games are Rimworld, Minecraft, Age of Empires, 7 Days to Die"

### Run Survey Simulation

In [60]:
# Run Simulation
##################################
SIMULATION_NAMEID = "video-game_airidas_dynamic_3"
LIMIT = None # For testing purposes. Set to NONE to run all
AUTO_INFO = {
    "date": bu.get_timestamp(),
    "EMBEDDING_NAMEID": EMBEDDING_NAMEID,
    "RETRIEVAL_PROMPT": RETRIEVAL_PROMPT,
    "CHUNKS_COUNT_IN_CTX": CHUNKS_COUNT_IN_CTX,
    "survey_type": str(type(surv)),
    "prompt_count": min(len(final_prompts), LIMIT) if LIMIT != None else len(final_prompts),
    "avg_tokens_in_prompt": round(prompt_info["total_all_prompt_tokens"]/len(final_prompts)),
    "prompt_template": prompt_template,
}
SETTINGS = {
    "model": "gpt-3.5-turbo",
    "temperature": 0.5,
}

##################################

import pandas as pd
from openai import OpenAI
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

completions = []
l = len(final_prompts)

for i, (prompt, question) in enumerate(list(zip(final_prompts, surv.questions))):
    if LIMIT != None and i > LIMIT:
        break

    print(f"{i}/{l}...", end="\t") # Print progress
    # Send the Request
    full_response = client.chat.completions.create(
        messages=prompt,
        model=SETTINGS["model"],
        temperature=SETTINGS["temperature"],
    )
    r = full_response.choices[0].message.content

    completions.append({'question': question, 'answer': r})

    print(f"{question}: {r}")

# Save results
df = pd.DataFrame(completions)
df.to_csv(f"simulations/{SIMULATION_NAMEID}_simulation.csv", index=False)
bu.quickJSON({"settings": SETTINGS, "info": AUTO_INFO}, f"simulations/{SIMULATION_NAMEID}_info.json")
bu.quickJSON(final_prompts, f"ignorefolder/simulations/{SIMULATION_NAMEID}_prompts.json")

0/40...	

What would you say if there were options to design your own avatar?: I LIKE IT
1/40...	What would you say if there were NO options to design your own avatar?: I EXPECT IT
2/40...	What would you say if the game had the option to save the game at any time?: I LIKE IT
3/40...	What would you say if the game did NOT have the option to save the game at any time?: I CAN TOLERATE IT
4/40...	What would you say if the game has good graphics?: I LIKE IT
5/40...	What would you say if the game had NO good graphics, or rather poor graphics?: I CAN TOLERATE IT
6/40...	What would you say if the game had an exciting storyline?: I LIKE IT
7/40...	What would you say if the game did NOT have an exciting storyline?: I CAN TOLERATE IT
8/40...	What would you say if there were rewards such as extra points, in-game currency or coins in the game?: I LIKE IT
9/40...	What would you say if there were NO rewards such as extra points, in-game currency or coins in the game?: I EXPECT IT
10/40...	What would you say if

#### Debug

## Analysis - General

### Load Simulation File (optional)

In [21]:
SIMULATION_NAMEID = "airi_kano_v2_dynamic_llama3_1"
DIRECTORY = "batch/output/_kano"
DIRECTORY_INFO = "batch/output/info"

df = pd.read_csv(os.path.join(DIRECTORY ,f'{SIMULATION_NAMEID}_simulation.csv'))
with open(os.path.join(DIRECTORY_INFO,f'{SIMULATION_NAMEID}_info.json'), 'r') as f:
    loaded = json.load(f)
try:
    AUTO_INFO = loaded["info"]
    SETTINGS = loaded["settings"]
    print("Settings and info loaded:")
    for k, v in AUTO_INFO.items():
        print(f"{k}: {v}")
    for k, v in SETTINGS.items():
        print(f"{k}: {v}")
except:
    print("No settings and/or info found")


try:
    if str(type(surv)) != AUTO_INFO["survey_type"]:
        print(f"WARNING: surv variable is not of the same type. {str(type(surv))} != {AUTO_INFO['survey_type']}")
except:
    pass

df.head(n=10)

Settings and info loaded:
date: 2024-04-22_173956
note: for dynamic, lower CHUNKS_COUNT_IN_CTX is used due to a greated chunk size variance to fit llamas max context window (12 vs 15)
EMBEDDING_NAMEID: airidas_reversed
RETRIEVAL_PROMPT: video game features
CHUNKS_COUNT_IN_CTX: 12
SUBJECT: airidas
prompt_count: 40
avg_tokens_in_prompt: 5822
prompt_template: systemMsg(   "You are specialized in impersonating people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit tastes by shadowing chats between the subject and friends. You will be asked to answer questions from the point of view of the persona. Text below:",   "Conversations between persona and friends",      "
NEW CONVERSATION:
".join(chunks_most_similar)  ),  # Understanding affirmation  assistantMsg('I will answer from the point of view of the persona, based on what I could the deduct from the text provided.'),  # Survey question. With Simulatio

,question,answer
0,What would you say if there were options to de...,I LIKE IT
1,What would you say if there were NO options to...,I DISLIKE IT
2,What would you say if the game had the option ...,I LIKE IT
3,What would you say if the game did NOT have th...,I DISLIKE IT
4,What would you say if the game has good graphics?,I LIKE IT
5,What would you say if the game had NO good gra...,I CAN TOLERATE IT
6,What would you say if the game had an exciting...,I LIKE IT
7,What would you say if the game did NOT have an...,I DISLIKE IT
8,What would you say if there were rewards such ...,I LIKE IT
9,What would you say if there were NO rewards su...,I CAN TOLERATE IT


In [26]:
surv = survey.KanoSurvey()

Using default Kano Survey CSV file: surveys/survey_kano-model.csv


### Cleanup

#### For GPT3.5

In [61]:
# remove all characters from a black list from the column answer
for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
    df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))

df['answer'] = df['answer'].str.upper()
# Update isValid
df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# if all values in isValid is true, drop the column, else print a message
if df['isValid'].all():
    df = df.drop('isValid', axis=1)
    print("All answers were valid")
else:
    print("Some answers were not valid")

df.head(n=10)

All answers were valid


,question,answer
0,What would you say if there were options to de...,I LIKE IT
1,What would you say if there were NO options to...,I EXPECT IT
2,What would you say if the game had the option ...,I LIKE IT
3,What would you say if the game did NOT have th...,I CAN TOLERATE IT
4,What would you say if the game has good graphics?,I LIKE IT
5,What would you say if the game had NO good gra...,I CAN TOLERATE IT
6,What would you say if the game had an exciting...,I LIKE IT
7,What would you say if the game did NOT have an...,I CAN TOLERATE IT
8,What would you say if there were rewards such ...,I LIKE IT
9,What would you say if there were NO rewards su...,I EXPECT IT


#### For Local Models

In [27]:
all_possible_asnwers = {}
all_possible_asnwers['POSSIBLE_ANSWERS'] = ["I LIKE IT", "I EXPECT IT", "I AM NEUTRAL", "I CAN TOLERATE IT", "I DISLIKE IT", "SOMEWHAT DISAGREE", "DISAGREE", "NEUTRAL", "SOMEWHAT AGREE", "AGREE"]

def extract_possible_answer(value):
    for phrase in all_possible_asnwers['POSSIBLE_ANSWERS']:
        pattern = r'(?i)' + re.escape(phrase)
        match = re.search(pattern, value)
        if match:
            return match.group()
    return value  # Return the original value if no possible answer is found

############ Invalid Answers ##################
def get_invalid_answers(value):
    if pd.isna(value):
        return ""
    elif value == "NaN":
        return ""
    elif value in all_possible_asnwers['POSSIBLE_ANSWERS']:
        return ""
    else:
        return value

In [29]:
df['answer'] = df['answer'].apply(lambda x: x.strip())
for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
    df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))
df['answer'] = df['answer'].str.upper()
df['answer'] = df['answer'].apply(extract_possible_answer)

# Update isValid
df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# if all values in isValid is true, drop the column, else print a message
if df['isValid'].all():
    df = df.drop('isValid', axis=1)
    print("All answers were valid")
else:
    print("Some answers were not valid")

All answers were valid


In [ ]:
# remove all strings from a black list from the column answer
for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
    df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))

df['answer'] = df['answer'].str.upper()
# Update isValid
df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# if all values in isValid is true, drop the column, else print a message
if df['isValid'].all():
    df = df.drop('isValid', axis=1)
    print("All answers were valid")
else:
    print("Some answers were not valid")

df.head(n=10)

### Proces simulation output - KANO

In [62]:
# Add airidas and elias answers
air = surv.test_answers["airidas"]
eli = surv.test_answers["elias"]
df.insert(2, "airidas", air[:len(df)])
df.insert(3, "elias", eli[:len(df)])

df['answer'] = df['answer'].str.upper()
df['airidas'] = df['airidas'].str.upper()
df['elias'] = df['elias'].str.upper()
df.head(n=10)

,question,answer,airidas,elias
0,What would you say if there were options to de...,I LIKE IT,I AM NEUTRAL,I AM NEUTRAL
1,What would you say if there were NO options to...,I EXPECT IT,I CAN TOLERATE IT,I EXPECT IT
2,What would you say if the game had the option ...,I LIKE IT,I EXPECT IT,I EXPECT IT
3,What would you say if the game did NOT have th...,I CAN TOLERATE IT,I DISLIKE IT,I CAN TOLERATE IT
4,What would you say if the game has good graphics?,I LIKE IT,I LIKE IT,I LIKE IT
5,What would you say if the game had NO good gra...,I CAN TOLERATE IT,I AM NEUTRAL,I CAN TOLERATE IT
6,What would you say if the game had an exciting...,I LIKE IT,I LIKE IT,I LIKE IT
7,What would you say if the game did NOT have an...,I CAN TOLERATE IT,I CAN TOLERATE IT,I DISLIKE IT
8,What would you say if there were rewards such ...,I LIKE IT,I LIKE IT,I DISLIKE IT
9,What would you say if there were NO rewards su...,I EXPECT IT,I DISLIKE IT,I LIKE IT


#### Proces simulation output - PERSONALITY

In [94]:
# Add airidas and elias answers
air = surv.test_answers["airidas"]
eli = surv.test_answers["elias"]
df.insert(2, "airidas", air[:len(df)])
df.insert(3, "elias", eli[:len(df)])
df.head(n=10)

,question,answer,airidas,elias
0,I am the life of the party.,AGREE,3,3
1,I don't talk a lot.,SOMEWHAT AGREE,2,2
2,I feel comfortable around people.,AGREE,4,4
3,I keep in the background.,SOMEWHAT AGREE,2,4
4,I start conversations.,AGREE,4,3
5,I have little to say.,DISAGREE,1,1
6,I talk to a lot of different people at parties.,AGREE,5,4
7,I don't like to draw attention to myself.,SOMEWHAT AGREE,2,3
8,I don't mind being the center of attention.,SOMEWHAT DISAGREE,4,1
9,I am quiet around strangers.,AGREE,1,4


### Remaps - UNIVERSAL

In [63]:
if isinstance(surv, survey.KanoSurvey):
    remap_dict = {"I EXPECT IT": 5, "I LIKE IT": 4, "I AM NEUTRAL": 3, "I CAN TOLERATE IT": 2, "I DISLIKE IT": 1}
    df['answer'] = df['answer'].map(remap_dict)
    df['airidas'] = df['airidas'].map(remap_dict)
    df['elias'] = df['elias'].map(remap_dict)
elif isinstance(surv, survey.PersonalitySurvey):
    remap_dict = {"AGREE": 5, "SOMEWHAT AGREE": 4, "NEUTRAL": 3, "SOMEWHAT DISAGREE": 2, "DISAGREE": 1}
    df['answer'] = df['answer'].map(remap_dict)

df.head()

,question,answer,airidas,elias
0,What would you say if there were options to de...,4,3,3
1,What would you say if there were NO options to...,5,2,5
2,What would you say if the game had the option ...,4,5,5
3,What would you say if the game did NOT have th...,2,1,2
4,What would you say if the game has good graphics?,4,4,4


In [ ]:
# compute one number of how the percentage of correct answers
result_data = {
    "Exact Matches": (df['answer'] == df['airidas']).sum() / len(df),
    "Correlation": df['answer'].corr(df['airidas']),
    "Exact Matches - elias": (df['answer'] == df['elias']).sum() / len(df),
    "Correlation - elias": df['answer'].corr(df['elias']),
}

for k, v in result_data.items():
    print(f"{k}: {v}")


#### Load Results CSV

In [ ]:
bu.if_dir_not_exist_make("results")
res = bu.LiveCSV("results/airidas_results.csv")

In [65]:
new_res = {
    # "label": None,
    "SIMULATION_NAMEID": SIMULATION_NAMEID,
    "timestamp": bu.get_timestamp(),
    "survey_type": str(type(surv)),
    "temperature": SETTINGS["temperature"],
    # "note": "",
    "exact_matches": result_data["Exact Matches"],
    "corr": result_data["Correlation"],
    "exact_matches_elias": result_data["Exact Matches - elias"],
    "corr_elias": result_data["Correlation - elias"],
}

tmp = bu.convert_dicts_to_table([new_res])
res.append_data(tmp[1], tmp[0])

brikasutils.quickCSV: Saved 16 as results/airidas_results.csv


## Batch Processing - Run Assembly

In [51]:
LIMIT = None
AUTO_INFO = {
    "EMBEDDING_NAMEID": EMBEDDING_NAMEID,
    "RETRIEVAL_PROMPT": RETRIEVAL_PROMPT,
    "CHUNKS_COUNT_IN_CTX": CHUNKS_COUNT_IN_CTX,
    "prompt_count": min(len(final_prompts), LIMIT) if LIMIT != None else len(final_prompts),
    "avg_tokens_in_prompt": round(prompt_info["total_all_prompt_tokens"]/len(final_prompts)),
    "prompt_tokens_min": round(prompt_info["prompt_tokens_min"]),
    "prompt_tokens_max": round(prompt_info["prompt_tokens_max"]),
}


bu.quickJSON(AUTO_INFO, f"ignorefolder/temp_run_assembly.json")
from pprint import pprint
pprint(AUTO_INFO)
print(type(surv))

{'CHUNKS_COUNT_IN_CTX': 12,
 'EMBEDDING_NAMEID': 'airidas_reversed',
 'RETRIEVAL_PROMPT': 'openess conciousness extrovert aggreableness neuroticism',
 'avg_tokens_in_prompt': 94,
 'prompt_count': 50,
 'prompt_tokens_max': 99,
 'prompt_tokens_min': 91}
<class 'survey.PersonalitySurvey'>


In [47]:
prompt_info

{'total_all_prompt_tokens': 355416,
 'prompt_tokens_min': 4381,
 'prompt_tokens_max': 12630}

## Batch processing (can be run without ones above)

In [57]:
import importlib
import pandas as pd
from openai import OpenAI
import os
import json
import traceback
import sys

import brikasutils as bu
import shared_utils as utils
import survey
importlib.reload(bu)
importlib.reload(utils)
importlib.reload(survey)

# SAVE_SIMULATION_DIR = "batch/output"
SAVE_SIMULATION_DIR = "batch/output/gpt35"
queue = bu.FileRunQueue(queue_folder_path="batch/queue", completed_folder_path="batch/done/gpt35")
report_live_csv = bu.LiveCSV("batch/run_reports_openai.csv")
timer = bu.Benchmarker()


for filepath in queue:
    timer.mark_start(filepath)

    try: 
        ########## Handle batch stuff ########
        path_csv = os.path.splitext(os.path.basename(filepath))[0]
        with open(filepath, 'r') as f:
            rundata = json.load(f)

        # Load prompt file
        with open(rundata["instructions"]["prompt_file"], 'r') as f:
            final_prompts = json.load(f)

        # Make the surv
        if rundata["instructions"]["survey_type"] == "KanoSurvey":
            surv = survey.KanoSurvey()
        elif rundata["instructions"]["survey_type"] == "PersonalitySurvey":
            surv = survey.PersonalitySurvey()
        else:
            raise Exception("Invalid survey type")

        timestamp = bu.get_timestamp()
        ######### Run Simulation ########
        SIMULATION_NAMEID = path_csv
        LIMIT = rundata["instructions"]["LIMIT"] if "LIMIT" in rundata["instructions"] else None
        AUTO_INFO = {
            "date": timestamp,
            **rundata["info"], # unpacked from rundata
            "limit": LIMIT,
            "prompt_count": min(len(final_prompts), LIMIT) if LIMIT != None else len(final_prompts),
            "avg_tokens_in_prompt": round(utils.describe_prompts(final_prompts)["total_all_prompt_tokens"]/len(final_prompts)),
        }
        SETTINGS = {
            **rundata["settings"], # unpacked from rundata
        }

        # client depends on if it's local or not
        if rundata["instructions"]["isLocal"]:
            client = OpenAI(
                base_url = 'http://localhost:11434/v1',
                api_key='ollama', # required, but unused
            )
        else:
            client = OpenAI(
                api_key=os.environ.get("OPENAI_API_KEY"),
            )

        completions = []
        l = len(final_prompts)

        for i, (prompt, question) in enumerate(list(zip(final_prompts, surv.questions))):
            if LIMIT != None and i > LIMIT:
                break

            print(f"{i}/{l}...", end="\t") # Print progress
            # Send the Request
            full_response = client.chat.completions.create(
                messages=prompt,
                **SETTINGS,
            )
            r = full_response.choices[0].message.content

            completions.append({'question': question, 'answer': r})

            print(f"{question}: {r}")
            
        ############ Save Important results
        df = pd.DataFrame(completions)
        bu.if_dir_not_exist_make(SAVE_SIMULATION_DIR)
        df.to_csv(f"{SAVE_SIMULATION_DIR}/{SIMULATION_NAMEID}_simulation.csv", index=False)
        bu.if_dir_not_exist_make(os.path.join(SAVE_SIMULATION_DIR, "info"))
        bu.quickJSON({"settings": SETTINGS, "info": AUTO_INFO}, f"{SAVE_SIMULATION_DIR}/info/{SIMULATION_NAMEID}_info.json")

        status = "OK"
    
    except Exception:
        print(f"##### Error while running {path_csv}.")
        error_string = traceback.format_exc()
        print(error_string)
        status = "Failed"

    ########### Time the run
    try:
        time_taken = timer.mark_end(filepath)
    except:
        print("Error while timing run: ")
        print(traceback.format_exc())
        time_taken = None

    ########### Report the run
    try:
        new_report = {
            "filename": path_csv,
            "timestamp": timestamp,
            "time_taken": time_taken,
            "status": status,
            **rundata["instructions"],
            "error": error_string if status == "Failed" else "",
        }

        tmp = bu.convert_dicts_to_table([new_report])
        report_live_csv.append_data(tmp[1], tmp[0])
    except Exception as e:
        print(f"Error while reporting: ")
        traceback.print_exc()

    print(f"Processed {path_csv}. Stauts: {status}")

timer.print_total_execution_time()

LiveCSV: File /Users/twenythree/Home/Prot/Thesis/batch/run_reports_openai.csv not existing. Creating new.
brikasutils.quickCSV: Saved 0 as batch/run_reports_openai.csv
[1/18] Running base_kano_v2_llama3_3.json from queue. 
Using default Kano Survey CSV file: surveys/survey_kano-model.csv
0/40...	What would you say if there were options to design your own avatar?: I LIKE IT
1/40...	What would you say if there were NO options to design your own avatar?: I DISLIKE IT.
2/40...	What would you say if the game had the option to save the game at any time?: I LIKE IT
3/40...	What would you say if the game did NOT have the option to save the game at any time?: I DISLIKE IT
4/40...	What would you say if the game has good graphics?: I LIKE IT
5/40...	What would you say if the game had NO good graphics, or rather poor graphics?: I CAN TOLERATE IT.
6/40...	What would you say if the game had an exciting storyline?: I LIKE IT
7/40...	What would you say if the game did NOT have an exciting storyline?: 

KeyboardInterrupt: 

## WIP

In [23]:
import persona
importlib.reload(persona)
import Thesis.standard_msg_reader as standard_msg_reader
importlib.reload(standard_msg_reader)
import whatsapp_msg_reader
importlib.reload(whatsapp_msg_reader)

<module 'whatsapp_msg_reader' from '/Users/twenythree/Home/Prot/Thesis/whatsapp_msg_reader.py'>

In [24]:
msgs = whatsapp_msg_reader.get_messages_from_JSONs(["selected-data/elias-wa/messages_1000.json"])

Read 659 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2023-03-12 to 2024-02-16


In [55]:
# combine two folder names in a path in a safe way
os.path.join(SAVE_SIMULATION_DIR, "info")

'batch/output/info'

['I LIKE IT', 'I EXPECT IT', 'I AM NEUTRAL', 'I CAN TOLERATE IT', 'I DISLIKE IT']


In [33]:
## Make V2
import csv

# Map all the file names recursively
paths_csvs = []
for dirpath, dirnames, files in os.walk('analysis/simulations-v2/to-convert-from-v1/csvs'):
    for file in files:
        full_path = os.path.join(dirpath, file)
        if os.path.isfile(full_path):
            paths_csvs.append(os.path.join( dirpath, file))


paths_infos = []
for dirpath, dirnames, files in os.walk('analysis/simulations-v2/to-convert-from-v1/infos'):
    for file in files:
        full_path = os.path.join(dirpath, file)
        if os.path.isfile(full_path):
            paths_infos.append(os.path.join( dirpath, file))


In [34]:
csv_info_pairs_paths = []
for path_csv in paths_csvs:
    if path_csv.endswith('.csv'):
        match = re.search(r'/([^/]+)_simulation\.csv', path_csv)
        if match:
            SIMULATION_NAMEID = match.group(1)
        else:
            raise Exception("Invalid csv file name: " + path_csv)
        
        for path_info in paths_infos:
            if SIMULATION_NAMEID in path_info:
                csv_info_pairs_paths.append((path_csv, path_info, SIMULATION_NAMEID))
    

In [35]:
bu.if_dir_not_exist_make("analysis/simulations-v2")
for csv_file, info_file, sim_id in csv_info_pairs_paths:
    sim_v2 = {}
    with open(info_file, 'r') as f:
        sim_v2["info"] = json.load(f)
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        sim_v2["QA"] = list(reader)
    # save the new simulation
    bu.quickJSON(sim_v2, f"analysis/simulations-v2/{sim_id}.json")

In [28]:
path = "analysis/simulations-v2/eli_pers_static-IMPERSONATE_2_llama3_70b.json"

# load the json and then put the key QA into a pandas df
with open(path, 'r') as f:
    sim = json.load(f)
    df = pd.DataFrame(sim["QA"])
    INFO = sim["info"]

In [31]:
INFO

{'settings': {'model': 'llama3:70b', 'timeout': 300},
 'info': {'date': '2024-04-30_021802',
  'CHUNKS_COUNT_IN_CTX': 10,
  'EMBEDDING_ID': '40-10',
  'VERSION_ID': '8k_pers',
  'model': 'nomic-embed-text',
  'CHUNK_SIZE': 40,
  'OVERLAP_SIZE': 10,
  'chunks_count': 138,
  'total_messages': 4465,
  'stat_total_msgs_in_chunks': 5520,
  'limit': None,
  'prompt_count': 50,
  'avg_tokens_in_prompt': 7029}}

In [32]:
if "prompt_count" in INFO["info"]:
    if INFO["info"]["prompt_count"] == 50:
        pass
    elif INFO["info"]["prompt_count"] == 40:
        pass
    else:
        print("Unknown prompt count")
elif "survey_type" in INFO["info"]:
    if INFO["info"]["survey_type"] == "KanoSurvey":
        pass
    elif INFO["info"]["survey_type"] == "PersonalitySurvey":
        pass
    else:
        print("Unknown survey type")
else:
    print("No survey indication")


In [33]:
# Load the simulation results
# both personality and game WIP

dfs = []
paths_csvs = []
# changed_values_per_column = {}

# List of folder paths
# folder_paths = ['simulations/local/video-game/', 'simulations/local/personality/']
# folder_paths = ['batch/output/_pers']
folder_paths = ['batch/output/llama3_70b/kano']#, 'batch/output/_pers']
# pers_folder_path = ['batch/output/_pers']
# kano_folder_path = ['batch/output/_kano']

# Read the first CSV file from the first folder to get the 'question' column
first_folder_path = folder_paths[0]
first_file_path = os.path.join(first_folder_path, os.listdir(first_folder_path)[0])
first_df = pd.read_csv(first_file_path)
questions = first_df['question']
# csv_loop_counter = 0

simulation_json = {
    "QA": [
        
    ],
    "info": []
}

for folder_path in folder_paths:
    for path_csv in os.listdir(folder_path):
        if path_csv.endswith('.csv'):
            path_csv = os.path.join(folder_path, path_csv)
            df = pd.read_csv(path_csv)

            # Store the original 'answer' column
            # original_answers = df['answer'].copy()
            
            # # remove all characters from a black list from the column answer
            df['answer'] = df['answer'].apply(lambda x: x.strip())
            for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
                df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))
            df['answer'] = df['answer'].str.upper()
            df['answer'] = df['answer'].apply(extract_possible_answer)
            # df['answer'] = df['answer'].apply(get_invalid_answers)
            
            filename_without_ext = path_csv[:-4]  # Remove the '.csv' extension
            filename_without_simulation = (
                filename_without_ext.replace('_simulation', '')
                                    # .replace('video-', '')
            )  # Remove '_simulation' from the filename
            paths_csvs.append(filename_without_simulation)
            dfs.append(df['answer'])
            # csv_loop_counter += 1

answers_df = pd.concat(dfs, axis=1, keys=paths_csvs)
# answers_df.insert(0, 'question', questions)  # Insert the 'question' column at the beginning
#print(f"Values changed: {changed_format}")
df = answers_df
df[:6]